In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
from helper_functions.clean_transitions import clean_transition_column
from helper_functions.simulate_game import simulate_game

In [3]:
avg_matrix = pd.read_excel('avg_transition_matrix_with_manual_edits.xlsx', sheet_name = 'Mean Pasted', index_col='Starting_State')

In [4]:
# def combine_matrices(matrix_A, matrix_B, avg, params=[]):
#     pass

In [5]:
from scipy.optimize import minimize

In [6]:
def accuracy(simulation_outcome, game_result):
    real_A_score = game_result.iloc[-1]['Home']
    real_B_score = game_result.iloc[-1]['Away']

    sim_A_score=[simulation_outcome[1][x][0] for x in range(len(simulation_outcome[1]))]
    sim_B_score=[simulation_outcome[1][x][1] for x in range(len(simulation_outcome[1]))]
    a_error = (real_A_score-sim_A_score)**2
    b_error = (real_B_score-sim_B_score)**2

    return np.mean(np.sqrt(a_error+b_error))

In [7]:
# abc=simulate_game(combined_matrix,num_games=100)

In [8]:
# #pseudo code
# for game in games:
#     load matrix_A
#     load matrix_B
#     find final score of game
#     run 100 or 1000 simulation of one possible param set
#     calculate accuracy

In [9]:
# Define optimization problem
def optimization_problem(scalar_multiples):
    # global m1, m2, m3, matrix1, matrix2, combined_matrix
    for g in tqdm(os.listdir('raw_game_pulls')):
        total_acc=0
        # game_res = pd.read_excel('raw_game_pulls/6195965_Play-By-Play for Indiana vs Army West Point.xlsx')
        game_res = pd.read_excel('raw_game_pulls/{}'.format(g))
        game_strings = g.split(' vs ')
        team_A = game_strings[0].split('for ')[-1]
        team_B = game_strings[1].split('.xlsx')[0]

        matrix1 = pd.read_excel('team_specific_matrix/{}_A.xlsx'.format(team_A), index_col='Starting_State')
        matrix2 = pd.read_excel('team_specific_matrix/{}_B.xlsx'.format(team_B), index_col='Starting_State')


        # Reshape the parameters into matrices
        m1 = scalar_multiples[:18].reshape(-1, 1)
        m2 = scalar_multiples[18:36].reshape(-1, 1)
        m3 = scalar_multiples[36:].reshape(-1, 1)

        # Construct combined matrix
        # combined_matrix = m1 * matrix1 + m2 * matrix2 + m3 * avg_matrix  # Adjust with scalar multiples
        combined_matrix = pd.DataFrame()
        for i in range(18):
            combined_matrix[i] = matrix1.iloc[:, i] * m1[i] + \
                                matrix2.iloc[:, i] * m2[i] + \
                                avg_matrix.iloc[:, i] * m3[i]
            
        combined_matrix = combined_matrix.div(combined_matrix.sum(axis=1), axis=0)
        combined_matrix.columns=avg_matrix.columns

        # Evaluate performance
        result = simulate_game(combined_matrix, 1)
        total_acc += accuracy(result, game_res)
    return np.mean(total_acc)

In [10]:
# Initialize parameters
# initial_guess = np.random.rand(54)
    
initial_guess=   [1,1,1,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,
                                0,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,
                                0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

# Define bounds for parameters (optional)
bounds = [(0, 1)] * 54  # Non-negative scalar multiples

In [13]:
result = minimize(optimization_problem, initial_guess, method='L-BFGS-B', bounds=bounds, options={'maxiter': 5})  # You can use different optimization methods

# Print optimal parameters and accuracy
print("Optimal scalar multiples:", result.x)
print("Optimal accuracy:", -result.fun)

In [94]:
from matplotlib import pyplot as plt

In [14]:
plt.bar(np.arange(0,54), result.x)

NameError: name 'plt' is not defined

In [75]:
combined_matrix=combined_matrix.rename(columns=)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Starting_State,,,,,,,,,,,,,,,,,,
Af0,0.309450,0.442764,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012479,0.000000,0.000000,0.000000,0.000000,0.000021,0.135323,0.000000,0.000000,0.099963
Af1,0.007299,0.016710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028628,0.000000,0.000000,0.000000,0.000000,0.000013,0.751433,0.000000,0.000000,0.195917
Af2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038453,0.000000,0.000000,0.000000,0.000000,0.000145,0.646094,0.000000,0.000000,0.315308
Af3,0.000000,0.000000,0.000417,0.000000,0.000000,0.000000,0.000000,0.000000,0.090334,0.000000,0.000000,0.000000,0.000000,0.000799,0.623798,0.000000,0.000000,0.284652
Ai0,0.111748,0.000000,0.017227,0.001875,0.065103,0.000000,0.000000,0.000000,0.216550,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.093661,0.081696,0.390862
Ai1,0.153233,0.000000,0.028436,0.002420,0.053316,0.000000,0.000000,0.000000,0.119816,0.000000,0.000000,0.000000,0.000000,0.015535,0.000000,0.098586,0.093312,0.435347
Ai2,0.149630,0.000000,0.030798,0.000380,0.066646,0.000000,0.000000,0.000000,0.095085,0.000000,0.000000,0.000000,0.000004,0.019024,0.000000,0.090581,0.097715,0.450137
Ai3,0.150781,0.000000,0.031008,0.002133,0.068067,0.000000,0.000000,0.000000,0.096527,0.000000,0.000000,0.000000,0.000000,0.015580,0.000000,0.096329,0.099605,0.439970
Ar0,0.172264,0.000000,0.027188,0.002079,0.072355,0.000000,0.000000,0.000000,0.100315,0.000000,0.000000,0.000000,0.000000,0.016685,0.000000,0.110367,0.094560,0.404188


In [16]:
avg_matrix

,Af0,Af1,Af2,Af3,Ai0,Ai1,Ai2,Ai3,Ar0,Bf0,Bf1,Bf2,Bf3,Bi0,Bi1,Bi2,Bi3,Br0
Starting_State,,,,,,,,,,,,,,,,,,
Af0,0.200006,0.546448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016108,0.000000,0.000000,0.000000,0.000000,0.000308,0.144473,0.000000,0.000000,0.092657
Af1,0.009219,0.028379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031846,0.000000,0.000000,0.000000,0.000000,0.000189,0.737337,0.000000,0.000000,0.193030
Af2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054198,0.000000,0.000000,0.000000,0.000000,0.002207,0.670735,0.000000,0.000000,0.272860
Af3,0.000000,0.000000,0.001401,0.000000,0.000000,0.000000,0.000000,0.000000,0.090947,0.000000,0.000000,0.000000,0.000000,0.010591,0.653435,0.000000,0.000000,0.243626
Ai0,0.063298,0.000000,0.012433,0.000901,0.062664,0.000000,0.000000,0.000000,0.254079,0.000000,0.000000,0.000000,0.000000,0.020880,0.000000,0.158314,0.070534,0.356896
Ai1,0.108310,0.000000,0.020689,0.001573,0.053738,0.000000,0.000000,0.000000,0.134363,0.000000,0.000000,0.000000,0.000000,0.019764,0.000000,0.173587,0.081052,0.406923
Ai2,0.093873,0.000000,0.017996,0.001017,0.060577,0.000000,0.000000,0.000000,0.115629,0.000000,0.000000,0.000000,0.000006,0.018748,0.000000,0.179339,0.093202,0.419613
Ai3,0.095393,0.000000,0.017591,0.001155,0.063860,0.000000,0.000000,0.000000,0.113292,0.000000,0.000000,0.000000,0.000000,0.019180,0.000000,0.180593,0.091781,0.417155
Ar0,0.108119,0.000000,0.021309,0.000914,0.068314,0.000000,0.000000,0.000000,0.120366,0.000000,0.000000,0.000000,0.000000,0.017005,0.000000,0.212585,0.082817,0.368571


In [62]:
avg_matrix.iloc[:, 18]

IndexError: single positional indexer is out-of-bounds

In [48]:
matrix1.iloc[:, i] * m1[i] + matrix2.iloc[:, i] * m2[i] + avg_matrix.iloc[:, i] * m3[i]

ValueError: ((18,), (1,))